In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType

import os

In [2]:
# change to be reflective of your environment
data_dir = '/home/cole/Workspace/School/Capstone/data/first_data_set/TestData/'

In [3]:
import os
size = os.path.getsize(data_dir+"/BACKUP_OBJECTS.csv")
end = 50 # how many gigs
upscale = int((end * 1073741824) / size)

In [4]:
# initiates a SparkContext which is necessary for accessing data in Spark
sc = SparkContext()
sqlContext = SQLContext(sc)
# change to match your environment
output_dir = data_dir + "/merge_data"

In [5]:
SDRO = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SD_RECON_ORDER.csv'])
SS_POOLS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir + '/SS_POOLS.csv'])
AFBF = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/AF_BITFILES.csv"])
BACKUP_OBJECTS = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/BACKUP_OBJECTS.csv"])
ls = [SDRO, SS_POOLS, AFBF, BACKUP_OBJECTS]

In [6]:
# upscale step
for i in range(len(ls)):
    csv = ls[i]
    concat = csv
    print(concat.count())
    for j in range(13):
        concat = concat.union(csv)
    ls[i] = concat
    print(concat.count())

53161928
744266992
113
1582
18048
252672
5823871
81534194


In [7]:
SDRO = ls[0]
SS_POOLS = ls[1]
AFBF = ls[2]
BACKUP_OBJECTS = ls[3]

In [8]:
full_outer_join = BACKUP_OBJECTS.join(AFBF, BACKUP_OBJECTS.OBJID == AFBF.BFID,how='left') # Could also use 'full_outer'
full_outer_join = full_outer_join.join(SDRO, ['OBJID'],how='left') # Could also use 'full_outer'

In [9]:
SDRO = None
SS_POOLS = None
AFBF = None
BACKUP_OBJECTS = None

In [10]:
df = full_outer_join
df = df.filter(df.POOLID. isNotNull())
df = df.withColumn("POOLID", df["POOLID"].cast("int"))
df = df.filter(df.ATTRLENGTH. isNotNull())
df = df.withColumn("ATTRLENGTH", df["ATTRLENGTH"].cast("float"))
df = df.filter(df.BFSIZE. isNotNull())
df = df.withColumn("BFSIZE", df["BFSIZE"].cast("float"))
df = df.filter(df.HDRSIZE. isNotNull())
df = df.withColumn("HDRSIZE", df["HDRSIZE"].cast("float"))
df = df.filter(df.OBJID. isNotNull())
df = df.withColumn("OBJID", df["OBJID"].cast("float"))

In [11]:
from pyspark.sql.functions import when   

df = df.withColumn('POOLID', when(df['POOLID'] == -1000000, 0).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == -9, 1).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 4, 2).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 6, 3).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 42, 4).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 72, 5).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == 82, 6).otherwise(df['POOLID']))
df = df.withColumn('POOLID', when(df['POOLID'] == -1, 7).otherwise(df['POOLID']))

In [12]:
df.select("OBJID", "ATTRLENGTH", "BFSIZE", "HDRSIZE", "POOLID").write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/merge_data/upscale")

In [13]:
df = None

In [14]:
sc.stop()

In [ ]:
# Dependencies
import glob, os
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
path = '/Users/zhaoluyang/Downloads/Senior-Capstone-2018-2019-master/Notebooks/TestData/merge_data/4_features'                     
path = data_dir + "/merge_data/4_features"
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_dataset   = pd.concat(df_from_each_file, ignore_index=True)

print("\nTraining a neural network on Iris dataset using TensorFlow ")
print("Loading the Iris data to memory...")
# Loading the dataset
# dataset = pd.read_csv('ourdata.csv')
print(concatenated_dataset)
print("Finish loading")

In [ ]:
# One-hot encoding for the categories
norma = pd.get_dummies(concatenated_dataset, columns=['POOLID']) 
values = list(concatenated_dataset.columns.values)
print(concatenated_dataset)

In [ ]:
#Normalization
X_train = concatenated_dataset[values[:4]]
X_train = ((X_train - X_train.min()) / (X_train.max() - X_train.min())).fillna(0)
print(X_train)


In [ ]:
#preprocess the data 
features = np.array(X_train, dtype='float32')
target = np.array(concatenated_dataset[values[4:]], dtype='float32')

# Shuffle Data
indices = np.random.choice(len(features), len(features), replace=False)
X_values = features[indices]
y_values = target[indices]

# Creating a Train and a Test Dataset
test_size = 1000
X_test = X_values[-test_size:]
X_train = X_values[:-test_size]
y_test = y_values[-test_size:]
y_train = y_values[:-test_size]

# print(X_train)
print(X_test)
# print(y_test)
# print(y_train)

In [ ]:
# define a neural network

# Initialize placeholders
X_data = tf.placeholder(shape=[None, 4], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 8], dtype=tf.float32)

#create seed for random_normal()
seed = 1234
np.random.seed(seed)
tf.set_random_seed(seed)

hidden_layer_nodes = 10
# We create a neural Network which contains 3 layers with 4, 8, 3 nodes repectively
w1 = tf.Variable(tf.random_normal(shape=[4,hidden_layer_nodes])) # Weight of the input layer
b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes]))   # Bias of the input layer
w2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes,8])) # Weight of the hidden layer
b2 = tf.Variable(tf.random_normal(shape=[8]))                    # Bias of the hidden layer
hidden_output = tf.nn.relu(tf.add(tf.matmul(X_data, w1), b1))
final_output = tf.nn.softmax(tf.add(tf.matmul(hidden_output, w2), b2))

# Loss Function
loss = tf.reduce_mean(-tf.reduce_sum(y_target * tf.log(final_output), axis=0))

# Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

print("A neural Network which contains 3 layers with 4, 10, 8 nodes repectively was created!")

In [ ]:
print('Training the model...')

# Interval / Epochs
interval = 100
epoch = 1000

# Initialize variables
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# Training the model...
for i in range(1, (epoch + 1)):
    sess.run(optimizer, feed_dict={X_data: X_train, y_target: y_train})
    if i % interval == 0:
        print('Epoch', i, '|', 'Loss:', sess.run(loss, feed_dict={X_data: X_train, y_target: y_train}))

print("Training finished\n")

In [ ]:
# get the accuracy of the model
correct_prediction = tf.equal(tf.argmax(final_output, 1), tf.argmax(y_target,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("The accuracy of the model is:", sess.run(accuracy, feed_dict={X_data: X_test, y_target: y_test}))

In [ ]:
# Prediction
np.set_printoptions(precision=4)
unknown = np.array([[0.693363, 0.0, 0.002894, 0.148097]], dtype=np.float32)
predicted = sess.run(final_output, feed_dict={X_data: unknown})
# model.predict(unknown)
print("Using model to predict pool id for features: ", unknown)
print("\nPredicted softmax vector is: ",predicted)
Class_dict={'POOLID_-1000000': 0, 'POOLID_-9': 1, 'POOLID_-1': 2, 'POOLID_4': 3, 'POOLID_-1': 4, 'POOLID_6': 5, 'POOLID_42': 6, 'POOLID_72': 7, 'POOLID_82': 8 }
pool_dict = {v:k for k,v in Class_dict.items()}
print("\nPredicted pool id is: ", pool_dict[np.argmax(predicted)])